---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [84]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [85]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [86]:
import re

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    towns = []
    with open('university_towns.txt', 'r') as fp:
        state = ''
        for row in fp:
            if('[edit]' in row):
                state = re.sub(r'\[.*', '', row).rstrip('\r\n')
            else:
                region_name = re.sub(r' \(.*', '', row).rstrip('\r\n')
                towns.append([state, region_name])
        
    return pd.DataFrame(towns, columns=['State', 'RegionName'])


In [87]:
gdp_values = pd.read_excel('gdplev.xls', skiprows=8, usecols=[4,6], header=None, names=['Quarter', 'GDP'])
gdp_values = gdp_values[gdp_values['Quarter'] >= '2000q1']
    
    
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    for i in range(2, len(gdp_values)):
        current_gdp = gdp_values.iloc[i]['GDP']
        n_minus_1_gdp = gdp_values.iloc[i-1]['GDP']
        n_minus_2_gdp = gdp_values.iloc[i-2]['GDP']
        if(current_gdp < n_minus_1_gdp and n_minus_1_gdp < n_minus_2_gdp):
            return gdp_values.iloc[i-1]['Quarter']
    

In [88]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_quarters = gdp_values[gdp_values['Quarter'] > get_recession_start()]
    
    for i in range(2, len(recession_quarters)):
        current_gdp = recession_quarters.iloc[i]['GDP']
        n_minus_1_gdp = recession_quarters.iloc[i-1]['GDP']
        n_minus_2_gdp = recession_quarters.iloc[i-2]['GDP']
        if(current_gdp > n_minus_1_gdp and n_minus_1_gdp > n_minus_2_gdp):
            return recession_quarters.iloc[i]['Quarter']

In [89]:
def get_recession_bottom():
    recession_quarters = gdp_values[gdp_values['Quarter'].between(get_recession_start(), get_recession_end())]
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    min_row_idx = recession_quarters['GDP'].idxmin()
    return recession_quarters.loc[min_row_idx]['Quarter']

'2009q2'

In [108]:


def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_by_quarters = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_by_quarters['State'] = housing_by_quarters['State'].map(states)
    housing_by_quarters = housing_by_quarters.set_index(['State', 'RegionName'])
    
    for year in range(2000, 2017):
        for quarter in range(0, 4):
            month_start = (quarter * 3) + 1
            column_names = list(map(lambda month: '{}-{:02d}'.format(year, month), range(month_start,  month_start + 3)))
            common_columns = housing_by_quarters.columns.intersection(column_names)
            if(len(common_columns) > 0):
                housing_by_quarters['{}q{}'.format(year, quarter + 1)] = housing_by_quarters[common_columns].mean(axis=1)
        
    return housing_by_quarters.filter(regex='\d{4}q\d')


202266.66666666666

In [118]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    housing_by_quarters = convert_housing_data_to_quarters()
    uni_towns = get_list_of_university_towns()
    
    housing_by_quarters['PriceRatio'] = housing_by_quarters[recession_start] / housing_by_quarters[recession_bottom]

    uni_towns_index = uni_towns.set_index(['State', 'RegionName']).index
    
    uni_town_prices = housing_by_quarters[housing_by_quarters.index.isin(uni_towns_index)]['PriceRatio']
    normal_town_prices = housing_by_quarters[~housing_by_quarters.index.isin(uni_towns_index)]['PriceRatio']

    s,p = ttest_ind(uni_town_prices.dropna(axis=0), normal_town_prices.dropna(axis=0))
    better = "university town" if uni_town_prices.mean() < normal_town_prices.mean() else "non-university town"
    
    return (p < 0.01, p, better)

run_ttest()

(True, 0.0054964273536938875, 'university town')